In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import tqdm
import os
import numpy as np
import skimage as ski

In [3]:
torch.manual_seed(12)

megaset_path = "/home/meribejayson/Desktop/SharkCNN/datasets-reduced/megaset/"
megaset_train_images_path = "/home/meribejayson/Desktop/SharkCNN/datasets-reduced/megaset/train/images/"
megaset_train_labels_path = "/home/meribejayson/Desktop/SharkCNN/datasets-reduced/megaset/train/labels/"

In [4]:
class LogisticRegresion(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        self.linear = nn.Linear(input_size,1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        
        return self.sig(x)

In [5]:
class SharkDataset(data.IterableDataset):

    def __init__(self):
        super(SharkDataset).__init__()
        self.image_names = os.listdir(megaset_train_images_path)
        self.num_images = len(self.image_names),
        self.curr_image_ordering = np.arange(self.num_images, dtype=np.int32)
        self.curr_image_idx = 0
        self.transform = A.Compose(
            [
               A.Blur(p=0.01, blur_limit=(3,7)),
               A.MedianBlur(p=0.01, blur_limit=(3,7)),
               A.ToGray(p=0.01),
               A.CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
            ]
        )

    def reset_random_image_ord(self):
        self.curr_image_order = np.random.shuffle(self.curr_image_ordering)
        self.curr_image_idx = 0

    """
        The following return a numpy array representing the image in BGR format
    """
    def get_random_image(self):
        if(self.curr_image_idx == self.num_images):
            self.reset_random_image_order()

        file_path = megaset_path + self.image_names[self.curr_image_order[self.curr_image_idx]]

        image = cv2.imread(file_path)

        self.curr_image_idx += 1

        return image

    def transform_image(self, image):
        return self.transform(image=image)["image"]
    
    """
        Assumes input image is in grayscale
    """
    def get_canny_output(self, image):
        return cv2.Canny(image, 100, 200)
    
    """
        Expecting image in BGR format
    """
    def get_color_gradients(self, image):
        blue_dy = ski.filters.sobel_h(image[:, :, 0])
        blue_dx = ski.filters.sobel_v(image[:, :, 0])

        green_dy = ski.filters.sobel_h(image[:, :, 1])
        green_dx = ski.filters.sobel_v(image[:, :, 1])

        red_dy = ski.filters.sobel_h(image[:, :, 2])
        red_dx = ski.filters.sobel_v(image[:, :, 2])

        return (blue_dx, blue_dy, green_dx, green_dy, red_dx, red_dy)
    
    def get_direction(self, image_dx, image_dy):
        return np.arctan(image_dy / image_dx)


    def __iter__(self):
        # worker_info = data.get_worker_info()

        # if worker_info == None:
        pass
